In [2]:
#!/usr/bin/env python
# coding: utf-8

# ## Task 2

#This is a script for testing for neutrinos reconstruction

import sys
#sys.path.append("/eos/home-a/acraplet/.local/lib/python2.7/site-packages")
sys.path.append("/home/acraplet/Alie/Masters/ICHiggsTauTauMasters/")
import uproot 
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from lbn_modified3 import LBN, LBNLayer
import tensorflow as tf


#for some reason pylorentz is installed somewhere differently ?
#sys.path.append("/eos/home-a/acraplet/.local/lib/python2.7/site-packages")
sys.path.append("/home/acraplet/Alie/Masters/ICHiggsTauTauMasters/")
from pylorentz import Momentum4
from pylorentz import Vector4
from pylorentz import Position4

# loading the tree
tree = uproot.open("/home/acraplet/Alie/Masters/MVAFILE_AllHiggs_tt.root")["ntuple"]
#tree = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_GluGluHToTauTauUncorrelatedDecay_Filtered_tt_2018.root")["ntuple"]
print("\n Tree loaded\n")



 Tree loaded



In [22]:
# define what variables are to be read into the dataframe
momenta_features = [ "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", #leading charged pi 4-momentum
              "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", #subleading charged pi 4-momentum
              "pi0_E_1","pi0_px_1","pi0_py_1","pi0_pz_1", #leading neutral pi 4-momentum
              "pi0_E_2","pi0_px_2","pi0_py_2","pi0_pz_2", #subleading neutral pi 4-momentum
              "gen_nu_p_1", "gen_nu_phi_1", "gen_nu_eta_1", #leading neutrino, gen level
              "gen_nu_p_2", "gen_nu_phi_2", "gen_nu_eta_2", #subleading neutrino, gen level  
              "pi2_E_1", "pi2_px_1", "pi2_py_1", "pi2_pz_1",
              "pi3_E_1", "pi3_px_1", "pi3_py_1", "pi3_pz_1"
                ] 

other_features = [ "ip_x_1", "ip_y_1", "ip_z_1",        #leading impact parameter
                   "ip_x_2", "ip_y_2", "ip_z_2",        #subleading impact parameter
                   #"y_1_1", "y_1_2",
                   "gen_phitt", "ip_sig_2", "ip_sig_1"
                 ]    # ratios of energies

target = [ "met", "metx", "mety", #"aco_angle_1", "aco_angle_6", "aco_angle_5", "aco_angle_7"
         ]  #acoplanarity angle
    
selectors = [ "tau_decay_mode_1","tau_decay_mode_2",
             "mva_dm_1","mva_dm_2","rand","wt_cp_ps","wt_cp_sm",
            ]

additional_info = [ "sv_x_1", "sv_y_1", "sv_z_1",
                    "sv_x_2", "sv_y_2", "sv_z_2",
                    ]

sv_covariance_matrices = ["svcov00_1", "svcov01_1", "svcov02_1",
                       "svcov10_1", "svcov11_1", "svcov12_1", 
                       "svcov20_1", "svcov21_1", "svcov22_1", 
                       "svcov00_2", "svcov01_2", "svcov02_2",
                       "svcov10_2", "svcov11_2", "svcov12_2", 
                       "svcov20_2", "svcov21_2", "svcov22_2", 
    
]

ip_covariance_matrices = ["ipcov00_1", "ipcov01_1", "ipcov02_1",
                       "ipcov10_1", "ipcov11_1", "ipcov12_1", 
                       "ipcov20_1", "ipcov21_1", "ipcov22_1", 
                       "ipcov00_2", "ipcov01_2", "ipcov02_2",
                       "ipcov10_2", "ipcov11_2", "ipcov12_2", 
                       "ipcov20_2", "ipcov21_2", "ipcov22_2", 
    
]

met_covariance_matrices = ["metcov00", 
                           "metcov01", 
                           "metcov10", 
                           "metcov11" ]

covs = sv_covariance_matrices + ip_covariance_matrices + met_covariance_matrices


variables4=(momenta_features+other_features+target+selectors+additional_info + covs) #copying Kinglsey's way cause it is very clean
print('Check 1')

df4 = tree.pandas.df(variables4)
# plt.figure()
# plt.hist2d(df4["mva_dm_1"],df4["mva_dm_2"], label = 'mva_dm_2 against mva_dm_1')
# plt.grid()
# #plt.legend()
# plt.savefig('The_mva_dm2d.png')

print('Check 1')

df4 = df4[
      (df4["tau_decay_mode_1"] == 10) 
    & (df4["tau_decay_mode_2"] == 1) 
    & (df4["mva_dm_1"] == 10) 
    & (df4["mva_dm_2"] == 1)
    & (df4["gen_nu_p_1"] > -4000)
    & (df4["gen_nu_p_2"] > -4000)
]

print(0.7*len(df4),'This is the length') #up to here we are fine

df_ps = df4[
      (df4["rand"]<df4["wt_cp_ps"]/2)     #a data frame only including the pseudoscalars
]

df_sm = df4[
      (df4["rand"]<df4["wt_cp_sm"]/2)     #data frame only including the scalars
]

print("panda Data frame created \n")

df4.head()

Check 1
Check 1
400190.0 This is the length
panda Data frame created 



,pi_E_1,pi_px_1,pi_py_1,pi_pz_1,pi_E_2,pi_px_2,pi_py_2,pi_pz_2,pi0_E_1,pi0_px_1,...,ipcov10_2,ipcov11_2,ipcov12_2,ipcov20_2,ipcov21_2,ipcov22_2,metcov00,metcov01,metcov10,metcov11
entry,,,,,,,,,,,,,,,,,,,,,
28,4.763911,4.132116,-2.350583,-0.275229,25.502320,-20.096222,13.177361,8.535096,0.0,0.0,...,1.034092e-06,0.000004,3.967108e-08,-8.706676e-07,3.967108e-08,0.000043,174.066971,-95.040672,-95.040672,79.905930
73,23.598206,8.073680,17.302803,13.866671,35.317532,5.329739,27.949494,-20.921956,0.0,0.0,...,-2.890363e-07,0.000002,3.899238e-07,-2.863723e-07,3.899238e-07,0.000009,2429.299561,678.568848,678.568848,2930.895020
102,53.494987,-41.784021,32.807944,-6.279217,6.952983,0.731218,4.313461,5.402210,0.0,0.0,...,-1.538588e-06,0.000002,-5.675425e-07,5.364135e-07,-5.675425e-07,0.000056,350.111694,-252.906754,-252.906754,327.191010
126,71.269803,9.985643,-16.439225,68.625099,10.564772,-6.779604,4.604359,6.665718,0.0,0.0,...,2.757374e-06,0.000006,-8.362060e-07,-1.138664e-07,-8.362060e-07,0.000035,78.484230,-52.514088,-52.514088,178.804672
145,27.890112,1.953204,-24.265267,13.609582,46.071747,20.033124,41.095961,5.690543,0.0,0.0,...,-5.879217e-07,0.000003,7.435281e-07,-3.501969e-07,7.435281e-07,0.000042,39.066010,29.357805,29.357805,248.615051


The above will be mega slow so we do not want to do it each time !

In [11]:
def cross_product(vector3_1,vector3_2):
    if len(vector3_1)!=3 or len(vector3_1)!=3:
        print('These are not 3D arrays !')
    x_perp_vector=vector3_1[1]*vector3_2[2]-vector3_1[2]*vector3_2[1]
    y_perp_vector=vector3_1[2]*vector3_2[0]-vector3_1[0]*vector3_2[2]
    z_perp_vector=vector3_1[0]*vector3_2[1]-vector3_1[1]*vector3_2[0]
    return np.array([x_perp_vector,y_perp_vector,z_perp_vector])

def dot_product(vector1,vector2):
    if len(vector1)!=len(vector2):
        raise Arrays_of_different_size
    prod=0
    for i in range(len(vector1)):
        prod=prod+vector1[i]*vector2[i]
    return prod


def norm(vector):
    if len(vector)!=3:
        print('This is only for a 3d vector')
    return np.sqrt(vector[0]**2+vector[1]**2+vector[2]**2)

def remove9999 (Momenta4, leading):
    if leading == 1:
        nu_ref = Momentum4.m_eta_phi_p(np.zeros(len(df4["gen_nu_phi_1"])), df4["gen_nu_eta_1"], df4["gen_nu_phi_1"], df4["gen_nu_p_1"])
    if leading == 2:
        nu_ref = Momentum4.m_eta_phi_p(np.zeros(len(df4["gen_nu_phi_2"])), df4["gen_nu_eta_2"], df4["gen_nu_phi_2"], df4["gen_nu_p_2"])
    
    array = np.array(Momenta4).T
    array = array[nu_ref.p_z != 9999]
    array = array.T
    return Momentum4(array[0], array[1], array[2], array[3])

    

In [12]:
#define the usefull 4 momenta

#neutrinos refs, in E, px, py, pz form
nu_1 = Momentum4.m_eta_phi_p(np.zeros(len(df4["gen_nu_phi_1"])), df4["gen_nu_eta_1"], df4["gen_nu_phi_1"], df4["gen_nu_p_1"])
nu_2 = Momentum4.m_eta_phi_p(np.zeros(len(df4["gen_nu_phi_2"])), df4["gen_nu_eta_2"], df4["gen_nu_phi_2"], df4["gen_nu_p_2"])

#Charged and neutral pion momenta
pi_1_4Mom = Momentum4(df4["pi_E_1"],df4["pi_px_1"],df4["pi_py_1"],df4["pi_pz_1"])
pi_2_4Mom = Momentum4(df4["pi_E_2"],df4["pi_px_2"],df4["pi_py_2"],df4["pi_pz_2"]) 
pi2_1_4Mom = Momentum4(df4["pi2_E_1"],df4["pi2_px_1"],df4["pi2_py_1"],df4["pi2_pz_1"])
pi3_1_4Mom = Momentum4(df4["pi3_E_1"],df4["pi3_px_1"],df4["pi3_py_1"],df4["pi3_pz_1"])

#Same for the pi0
pi0_1_4Mom = Momentum4(df4["pi0_E_1"],df4["pi0_px_1"],df4["pi0_py_1"],df4["pi0_pz_1"])
pi0_2_4Mom = Momentum4(df4["pi0_E_2"],df4["pi0_px_2"],df4["pi0_py_2"],df4["pi0_pz_2"])

tau_1_vis = pi_1_4Mom + pi3_1_4Mom + pi2_1_4Mom # + pi0_1_4Mom #
tau_2_vis = pi_2_4Mom + pi0_2_4Mom



In [34]:
hist1 = tau_1_vis.eta - nu_1.eta
hist2 = tau_2_vis.eta - nu_2.eta

hist1 = tf.where(tf.math.is_inf(hist1), 999, hist1)

plt.figure()
#plt.xlim(0,5)
plt.hist(hist1, alpha = 0.5, bins = 100, label='eta_vis_1(a1) - eta_nu1')
plt.hist(hist2, alpha = 0.5, bins = 100, label='eta_vis_1(a1) - eta_nu1')
plt.legend()
plt.grid()

plt.title('Colinearity approximation - eta')
plt.xlabel('Difference between eta visible and eta neutrinos (rad)')
plt.savefig('Eta_colinearity')
plt.close()

KeyboardInterrupt: 

In [13]:
tau_2 = tau_2_vis + nu_2
tau_1 = tau_1_vis + nu_1

plt.figure()
plt.hist(tau_1.m, alpha = 0.5, bins=100)
plt.hist(tau_2.m, alpha = 0.5,  bins=100)
plt.xlabel('Mass_GEV')
plt.xlim(0,4)
#plt.savefig('Is_this_a1.png')
plt.close()

/home/acraplet/miniconda3/envs/ICMasters/lib/python3.8/site-packages/numpy/lib/histograms.py:851: ComplexWarning: Casting complex values to real discards the imaginary part
  indices = f_indices.astype(np.intp)
/home/acraplet/miniconda3/envs/ICMasters/lib/python3.8/site-packages/numpy/core/_asarray.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/acraplet/miniconda3/envs/ICMasters/lib/python3.8/site-packages/matplotlib/transforms.py:1966: ComplexWarning: Casting complex values to real discards the imaginary part
  x, y = float(x), float(y)


### From Here, start straining to regress neutrinos

### defining loss function

In [14]:
# Remove the nans of the ip values:
# "ip_x_1", "ip_y_1", "ip_z_1",        #leading impact parameter
#                    "ip_x_2", "ip_y_2", "ip_z_2"
def one_d(val):
    return tf.constant(val, shape = df4["pi_px_1"].shape, dtype = np.float32)

def Mom4_to_tf(Mom4_1D):
    return tf.convert_to_tensor(Mom4_1D, dtype = 'float32')

ip_x_1 = tf.where(tf.math.is_nan(df4["ip_x_1"]), 0, df4["ip_x_1"])
ip_y_1 = tf.where(tf.math.is_nan(df4["ip_y_1"]), 0, df4["ip_y_1"])
ip_z_1 = tf.where(tf.math.is_nan(df4["ip_z_1"]), 0, df4["ip_z_1"])
ip_x_2 = tf.where(tf.math.is_nan(df4["ip_x_2"]), 0, df4["ip_x_2"])
ip_y_2 = tf.where(tf.math.is_nan(df4["ip_y_2"]), 0, df4["ip_y_2"])
ip_z_2 = tf.where(tf.math.is_nan(df4["ip_z_2"]), 0, df4["ip_z_2"])


sv_x_1 = tf.where(tf.math.is_nan(df4["sv_x_1"]), 0, df4["sv_x_1"])
sv_y_1 = tf.where(tf.math.is_nan(df4["sv_y_1"]), 0, df4["sv_y_1"])
sv_z_1 = tf.where(tf.math.is_nan(df4["sv_z_1"]), 0, df4["sv_z_1"])
sv_x_2 = tf.where(tf.math.is_nan(df4["sv_x_2"]), 0, df4["sv_x_2"])
sv_y_2 = tf.where(tf.math.is_nan(df4["sv_y_2"]), 0, df4["sv_y_2"])
sv_z_2 = tf.where(tf.math.is_nan(df4["sv_z_2"]), 0, df4["sv_z_2"])



### training 1

In [15]:
# Start training here: would work better with small dataset !
# Here copying the result from the KIT paper

smear_px, smear_py = one_d(0), one_d(0)   #the smearing of the detector, we don't know yet what it is

ref = [#smear_px,py                      #0
       #one_d(1.776),                      #1
       #df4["metx"],                   #2
       #df4["mety"],                   #3
       Mom4_to_tf(tau_1_vis.e),       #4
       Mom4_to_tf(tau_1_vis.p_x),     #5
       Mom4_to_tf(tau_1_vis.p_y),     #6
       Mom4_to_tf(tau_1_vis.p_z),     #7
       Mom4_to_tf(tau_2_vis.e),       #8
       Mom4_to_tf(tau_2_vis.p_x),     #9 
       Mom4_to_tf(tau_2_vis.p_y),     #10
       Mom4_to_tf(tau_2_vis.p_z),     #11
       #one_d(125),                    #12
       #Mom4_to_tf(nu_1.e),            #13       corresponding to          #0
       Mom4_to_tf(nu_1.p_x),          #14                                 #1
       Mom4_to_tf(nu_1.p_y),          #15                                 #2
       Mom4_to_tf(nu_1.p_z),          #16                                 #3
       #Mom4_to_tf(nu_2.e),            #17                                 #4
       Mom4_to_tf(nu_2.p_x),          #18                                 #5
       Mom4_to_tf(nu_2.p_y),          #19                                 #6
       Mom4_to_tf(nu_2.p_z),          #20                                 #7
]


#i_smear_px = 0
#i_tau_mass = 0
#i_smeared_met_px = 1
#i_smeared_met_py = 2
i_tau1_e = 0
i_tau1_px = 1
i_tau1_py = 2
i_tau1_pz = 3
i_tau2_e = 4
i_tau2_px = 5
i_tau2_py = 6
i_tau2_pz = 7
#i_gen_mass = 9
#i_nu1_e = 12
i_nu1_px = 8
i_nu1_py = 9
i_nu1_pz = 10
#i_nu2_e = 16
i_nu2_px = 11
i_nu2_py = 12
i_nu2_pz = 13

####Very important, the batch size must be defined beforehand to get rid of two trainable params !
B_size = 500#2**10

m_Higgs_squared = 125**2


m_tau_squared = 1.776**2

def one_d_traning(val, shape_array):
    return tf.constant(val, shape = shape_array.shape, dtype = np.float32)

ref = tf.transpose(ref)

def loss_D_p (y_true, y_pred):
    global ratio_all
    #calculting the difference between the the components, need to add the smearing of detector eventually
    target_components = [i_nu1_px, i_nu1_py, i_nu1_pz, i_nu2_px, i_nu2_py, i_nu2_pz]
    target_components_diff_list = []
    for i in target_components: target_components_diff_list.append((y_true[:,i]-y_pred[:,i])**2)
    dxyz = 0
    for d in target_components_diff_list: dxyz+=d
    return ratio_all * dxyz #tone it down cause it takes too much space

def energy_nu (y_true, y_pred, number):
    if number == 1:
        return tf.sqrt(y_pred[:, i_nu1_px]**2 + y_pred[:, i_nu1_py]**2 + y_pred[:, i_nu1_pz]**2)
    if number == 2:
        return tf.sqrt(y_pred[:, i_nu2_px]**2 + y_pred[:, i_nu2_py]**2 + y_pred[:, i_nu2_pz]**2)

def loss_mass_tau(y_true, y_pred):
    #now we try only to use the y_pred for neutrino info, this is I guess their way of 
    #only training for neutrino info whilst keeping nice structure
    #we are always assuming m=0
    # note, we are taking y_tau as exact, we could choose not to...
    global ratio_tau
    E1 = y_true[:, i_tau1_e] + energy_nu(y_true, y_pred, 1) 
    P1_squared = (y_true[:, i_tau1_px] + y_pred[:, i_nu1_px])**2 + (y_true[:, i_tau1_py] + y_pred[:, i_nu1_py])**2 + (y_true[:, i_tau1_pz] + y_pred[:, i_nu1_pz])**2
    R1 = (E1**2 - P1_squared -  m_tau_squared)/m_Higgs_squared
    
    E2 = y_true[:, i_tau2_e] + energy_nu(y_true, y_pred, 2) 
    P2_squared = (y_true[:, i_tau2_px] + y_pred[:, i_nu2_px])**2 + (y_true[:, i_tau2_py] + y_pred[:, i_nu2_py])**2 + (y_true[:, i_tau2_pz] + y_pred[:, i_nu2_pz])**2
    R2 = (E2**2 - P2_squared - m_tau_squared)/m_Higgs_squared
    return ratio_tau * (tf.math.abs(R1) + tf.abs(R2))


def loss_mass_Higgs(y_true, y_pred):
    global ratio_H
    print(tf.convert_to_tensor(y_pred[:, 0]).shape, 'this is shape')
    EH = y_true[:, i_tau1_e] + energy_nu(y_true, y_pred, 1) + y_true[:, i_tau2_e] + energy_nu(y_true, y_pred, 2)
    px_H = y_true[:, i_tau1_px] + y_true[:, i_tau2_px] + y_pred[:, i_nu1_px] + y_pred[:, i_nu2_px]
    py_H = y_true[:, i_tau1_py] + y_true[:, i_tau2_py] + y_pred[:, i_nu1_py] + y_pred[:, i_nu2_py]
    pz_H = y_true[:, i_tau1_pz] + y_true[:, i_tau2_pz] + y_pred[:, i_nu1_pz] + y_pred[:, i_nu2_pz]
#      y_true[:, i_gen_mass]**2
    return ratio_H* tf.abs((EH**2-px_H**2-py_H**2-pz_H**2 -m_Higgs_squared)/m_Higgs_squared)


def loss_phi(y_true, y_pred):
    global ratio_phi
    phi_diff_1 = (tf.math.atan2(y_pred[:, i_nu1_py],y_pred[:, i_nu1_px])-tf.math.atan2(y_true[:, i_nu1_py],y_true[:, i_nu1_px]))**2
    phi_diff_2 = (tf.math.atan2(y_pred[:, i_nu2_py],y_pred[:, i_nu2_px])-tf.math.atan2(y_true[:, i_nu2_py],y_true[:, i_nu2_px]))**2

    
    return ratio_phi*tf.convert_to_tensor(phi_diff_1+phi_diff_2) #tone it up


def loss_p(y_true, y_pred):
    global ratio_p
    delta_p1 = (energy_nu(y_true,y_pred, 1) - tf.sqrt(y_true[:, i_nu1_px]**2 + y_true[:, i_nu1_py]**2 + y_true[:, i_nu1_pz]**2))**2
    delta_p2 = (energy_nu(y_true,y_pred, 2) - tf.sqrt(y_true[:, i_nu2_px]**2 + y_true[:, i_nu2_py]**2 + y_true[:, i_nu2_pz]**2))**2
    
    return ratio_p*tf.convert_to_tensor(delta_p1+delta_p2) #tone it up
    



def loss_fn(y_true, y_pred):

    dxyz = loss_D_p(y_true, y_pred)
    #dmet = loss_dmet(y_true, y_pred)
    #dPTtaus = loss_dPTtaus(y_true, y_pred)
    dphi = loss_phi(y_true,y_pred)
    dtau = loss_mass_tau(y_true, y_pred)
    dHiggs = loss_mass_Higgs(y_true, y_pred)
    dp = loss_p(y_true, y_pred)
    #dM = loss_dM_had(y_true, y_pred)

    return dxyz + dtau + dHiggs + dphi + dp


nu_1_training = np.array(nu_1)[0]

print(nu_1_training.shape)

n = -1

x = np.array([
              #smear_px,
              #smear_py,
              df4["metx"],
              df4["mety"], 
              Mom4_to_tf(tau_1_vis.e),
              Mom4_to_tf(tau_1_vis.p_x),
              Mom4_to_tf(tau_1_vis.p_y),
              Mom4_to_tf(tau_1_vis.p_z),
              Mom4_to_tf(tau_2_vis.e),
              Mom4_to_tf(tau_2_vis.p_x),
              Mom4_to_tf(tau_2_vis.p_y),
              Mom4_to_tf(tau_2_vis.p_z),
              ip_x_1, 
              ip_y_1, 
              ip_z_1, 
              ip_x_2, 
              ip_y_2, 
              ip_z_2,
              df4["ip_sig_2"],
              df4["ip_sig_1"],
              #Mom4_to_tf(nu_1.p_x),
              #df4["ip_x_1"], 
              #df4["ip_y_1"], 
              #df4["ip_z_1"],        #leading impact parameter
              #df4["ip_x_2"], 
              #df4["ip_y_2"], 
              #df4["ip_z_2"],
              df4["met"],  
              sv_x_1, 
              sv_y_1, 
              sv_z_1,
              #sv_x_2, 
              #sv_y_2, 
              #sv_z_2
             ])
x = tf.transpose(x)

y = ref

ratio_tau = 10
ratio_H = 4
ratio_all = 1/10000
ratio_p = 10/1500
ratio_phi = 4


trainFrac = 0.7
numTrain = int(trainFrac*x.shape[0])
x_train = x[:numTrain]
y_train = y[:numTrain]

x_val = x[numTrain:]
y_val = y[numTrain:]

input_1 = tf.keras.Input(shape = x.shape, name="lab_frame")
x2 = tf.keras.layers.Dense(300, activation = 'relu', name="learning")(input_1)
#,kernel_regularizer=tf.keras.regularizers.L1L2(2)
#kernel_regularizer=tf.keras.regularizers.L2(0.2),
#x4 = tf.keras.layers.Dropout(0.5, name="dropout")(x2)
# x6 = tf.keras.layers.Dense(11, activation = 'relu', name="learning1")(x2)

#x5 = tf.keras.layers.Dropout(0.4, name="dropout")(x201
# x3 = tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.01), name="learning2")(x2)

x3 = tf.keras.layers.Dense(300, activation = 'relu', name="learning2")(x2)


# kernel_regularizer=tf.keras.regularizers.L1L2(2)
# kernel_regularizer=tf.keras.regularizers.L2(0.2),
x4 = tf.keras.layers.Dropout(0.1, name="dropout2")(x3)
#x4 = tf.keras.layers.Dense(300, activation = 'elu', name="learning3")(x3)
#x4 = tf.keras.layers.Dense(500, activation = 'relu', name="learning3")(x3)
output = tf.keras.layers.Dense(14, name="output")(x4)

model = tf.keras.Model(
    inputs=[input_1],
    outputs=[output],
)

y = ref


model.summary()

#loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(loss = loss_fn, optimizer = 'adam', metrics = ['mae', loss_D_p, loss_phi, loss_p, loss_mass_tau, loss_mass_Higgs])#, loss_mass_Higgs, loss_mass_tau, loss_D_p])

history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
    epochs=25,
    batch_size = B_size)


(571700,)
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lab_frame (InputLayer)       [(None, 571700, 22)]      0         
_________________________________________________________________
learning (Dense)             (None, 571700, 300)       6900      
_________________________________________________________________
learning2 (Dense)            (None, 571700, 300)       90300     
_________________________________________________________________
dropout2 (Dropout)           (None, 571700, 300)       0         
_________________________________________________________________
output (Dense)               (None, 571700, 14)        4214      
Total params: 101,414
Trainable params: 101,414
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on L

Epoch 16/25
801/801 [==============================] - 5s 6ms/step - loss: 15.3359 - mae: 35.6650 - loss_D_p: 0.1478 - loss_phi: 6.3706 - loss_p: 7.6435 - loss_mass_tau: 0.4065 - loss_mass_Higgs: 0.7676 - val_loss: 16.3155 - val_mae: 39.8327 - val_loss_D_p: 0.2003 - val_loss_phi: 4.1623 - val_loss_p: 10.8082 - val_loss_mass_tau: 0.3511 - val_loss_mass_Higgs: 0.7936
Epoch 17/25
801/801 [==============================] - 5s 6ms/step - loss: 15.6138 - mae: 35.8444 - loss_D_p: 0.1477 - loss_phi: 6.6441 - loss_p: 7.6336 - loss_mass_tau: 0.4209 - loss_mass_Higgs: 0.7676 - val_loss: 17.5212 - val_mae: 40.2167 - val_loss_D_p: 0.2043 - val_loss_phi: 5.0392 - val_loss_p: 11.0451 - val_loss_mass_tau: 0.4117 - val_loss_mass_Higgs: 0.8209
Epoch 18/25
801/801 [==============================] - 5s 6ms/step - loss: 16.2077 - mae: 35.9228 - loss_D_p: 0.1492 - loss_phi: 7.1816 - loss_p: 7.6568 - loss_mass_tau: 0.4450 - loss_mass_Higgs: 0.7751 - val_loss: 17.1468 - val_mae: 40.0016 - val_loss_D_p: 0.2041

In [21]:
#Here verify the eta component


def checks(x_array, length):
    
    regressed_array = model({"lab_frame": x_array})

    #length = 0#numTrain

    E_1 = norm([regressed_array[:, i_nu1_py],regressed_array[:, i_nu1_py], regressed_array[:, i_nu1_pz]])
    E_2 = norm([regressed_array[:, i_nu2_py],regressed_array[:, i_nu2_py], regressed_array[:, i_nu2_pz]])


    nu_1_regressed = Momentum4(E_1,regressed_array[:, i_nu1_px],regressed_array[:, i_nu1_py], regressed_array[:, i_nu1_pz])
    nu_2_regressed = Momentum4(E_2,regressed_array[:, i_nu2_px],regressed_array[:, i_nu2_py], regressed_array[:, i_nu2_pz])


    print('All means')

    hist1 = nu_1_regressed.p_x
    hist2 = Mom4_to_tf(nu_1.p_x)[length:]
    hist_a = np.array(hist2-hist1) #
    hist1 = nu_2_regressed.p_x
    hist2 = Mom4_to_tf(nu_2.p_x)[length:]
    hist_b = np.array(hist2-hist1) #
    hist1 = nu_1_regressed.p_y
    hist2 = Mom4_to_tf(nu_1.p_y)[length:]
    hist_c = np.array(hist2-hist1) #
    hist1 = nu_2_regressed.p_y
    hist2 = Mom4_to_tf(nu_2.p_y)[length:]
    hist_l = np.array(hist2-hist1) #
    hist1 = nu_1_regressed.p_z
    hist2 = Mom4_to_tf(nu_1.p_z)[length:]
    hist_d = np.array(hist2-hist1) #
    hist1 = nu_2_regressed.p_z
    hist2 = Mom4_to_tf(nu_2.p_z)[length:]
    hist_e = np.array(hist2-hist1) #
    hist1 = nu_1_regressed.phi
    hist2 = Mom4_to_tf(nu_1.phi)[length:]
    hist_f = np.array(hist2-hist1) #
    hist1 = nu_2_regressed.phi
    hist2 = Mom4_to_tf(nu_2.phi)[length:]
    hist_g = np.array(hist2-hist1) #
    hist1 = nu_1_regressed.eta
    hist2 = Mom4_to_tf(nu_1.eta)[length:]
    hist_h = np.array(hist2-hist1) #
    hist1 = nu_2_regressed.eta
    hist2 = Mom4_to_tf(nu_2.eta)[length:]
    hist_i = np.array(hist2-hist1) #
    hist1 = nu_1_regressed.p
    hist2 = Mom4_to_tf(nu_1.p)[length:]
    hist_j = np.array(hist2-hist1) #
    hist1 = nu_2_regressed.p
    hist2 = Mom4_to_tf(nu_2.p)[length:]
    hist_k = np.array(hist2-hist1) #

    return [hist_a.mean(), hist_b.mean(), hist_c.mean(), hist_d.mean(), hist_l.mean(), hist_e.mean(), 
            hist_f.mean(), hist_g.mean(), hist_h.mean(), hist_i.mean(), hist_j.mean(), hist_k.mean()], [hist_a.std(), hist_b.std(), hist_c.std(), hist_d.std(), hist_l.std(), hist_e.std(), 
            hist_f.std(), hist_g.std(), hist_h.mean(), hist_i.std(), hist_j.std(), hist_k.std()]

    

mean_x_val, std_x_val = checks(x_val, numTrain)
mean_x, std_x = checks(x, 0)


    
table = [mean_x_val, std_x_val, mean_x, std_x]

table = np.array(table).T

np.savetxt('check.txt', table, fmt = '%.2f')

print('done')


All means
All means
done


In [ ]:
# print('means')

# print('%.2f'%hist_a.mean())
# print('%.2f'%hist_b.mean())
# print('%.2f'%hist_c.mean())
# print('%.2f'%hist_d.mean())
# print('%.2f'%hist_b.mean())
# print('%.2f'%hist_e.mean())
# print('%.2f'%hist_f.mean())
# print('%.2f'%hist_g.mean())
# print('%.2f'%hist_h.mean())
# print('%.2f'%hist_i.mean())
# print('%.2f'%hist_j.mean())
# print('%.2f'%hist_k.mean())


# print ('stds')

# print('%.2f'%hist_a.std())
# print('%.2f'%hist_b.std())
# print('%.2f'%hist_c.std())
# print('%.2f'%hist_d.std())
# print('%.2f'%hist_b.std())
# print('%.2f'%hist_e.std())
# print('%.2f'%hist_f.std())
# print('%.2f'%hist_g.std())
# print('%.2f'%hist_h.std())
# print('%.2f'%hist_i.std())
# print('%.2f'%hist_j.std())
# print('%.2f'%hist_k.std())

In [ ]:
#Then Try simple NN to regress only the phi component of the two neutrinos simultaneously

x = [Mom4_to_tf(tau_1_vis.e),
     Mom4_to_tf(tau_1_vis.p_x),
     Mom4_to_tf(tau_1_vis.p_y),
     Mom4_to_tf(tau_1_vis.p_z),
     Mom4_to_tf(tau_1_vis.phi),
     Mom4_to_tf(tau_2_vis.e),
     Mom4_to_tf(tau_2_vis.p_x),
     Mom4_to_tf(tau_2_vis.p_y),
     Mom4_to_tf(tau_2_vis.p_z),
     Mom4_to_tf(tau_2_vis.phi),
     df4['met'],
     df4['metx'],
     df4['mety']
    ]
x = tf.transpose(x)

y = [Mom4_to_tf(nu_1.phi), Mom4_to_tf(nu_2.phi)]

y = tf.transpose(y)


# def loss_fn_phi(y_true, y_pred):
#     return (tf.math.cos(y_true)-tf.math.cos(y_pred))**2

input_1 = tf.keras.Input(shape = x.shape, name="lab_frame")

x2 = tf.keras.layers.Dense(64, activation = 'relu', name="learning")(input_1)
x3 = tf.keras.layers.Dense(64, activation = 'relu', name="learning2")(x2)
x4 = tf.keras.layers.Dropout(0.1, name="dropout")(x3)
#x4 = tf.keras.layers.Dense(300, activation = 'relu', name="learning3")(x3)
#x4 = tf.keras.layers.Dense(500, activation = 'relu', name="learning3")(x3)
#x5 = tf.keras.layers.Dense(500, activation = 'elu', name="learning4")(x4)
#x6 = tf.keras.layers.Dense(500, activation = 'relu', name="learning5")(x5)

output = tf.keras.layers.Dense(1, name="output")(x4)

model = tf.keras.Model(
    inputs=[input_1],
    outputs=[output],
)

# t = tau_1_vis_loss
# y_ = y.T

model.summary()

loss_fn_phi = tf.keras.losses.MeanSquaredError()
model.compile(loss = loss_fn_phi, optimizer = 'adam', metrics = ['mae'])#, loss_mass_Higgs, loss_mass_tau, loss_D_p])

history = model.fit(x, y, validation_split = 0.3,
    epochs=50,
    batch_size = 500)




In [40]:
#This is the 'quality check' section
fig = plt.figure(figsize=(10,10), frameon = True)

plt.title('Quality check: 300-300-300-o,\ninputs exclude mets, output exclude met. Batch: 500')
ax = fig.add_subplot(2,1,1)

hist1 = np.array(model({"lab_frame": x_val})[:, -6])
hist2 = np.array(y_val[:, -6])
hist_a = hist2-hist1
plt.hist(hist_a, bins = 100, alpha = 0.5,label = "True-Regressed nu_px_1\nMean: %.2f, std: %.2f"%(hist_a.mean(), hist_a.std()))

hist1 = np.array(model({"lab_frame": x_val})[:, -5])
hist2 = np.array(y_val[:, -5])
hist_b = hist2-hist1
plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_py_1\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))

hist1 = np.array(model({"lab_frame": x_val})[:, -4])
hist2 = np.array(y_val[:, -4])
hist_c = hist2-hist1
plt.hist(hist_c, bins = 100, alpha = 0.5,label = "True-Regressed nu_pz_1\nMean: %.2f, std: %.2f"%(hist_c.mean(), hist_c.std()))
plt.grid()
plt.legend()
plt.xlim(-50,50)
plt.ylabel('Occurences')
         
         
ax = fig.add_subplot(2,1,2)

hist1 = np.array(model({"lab_frame": x_val})[:, -3])
hist2 = np.array(y_val[:, -3])
hist_d = hist2-hist1
plt.hist(hist_d, bins = 100, alpha = 0.5,label = "True-Regressed nu_px_2\nMean: %.2f, std: %.2f"%(hist_d.mean(), hist_d.std()))

hist1 = np.array(model({"lab_frame": x_val})[:, -2])
hist2 = np.array(y_val[:, -2])
hist_e = hist2-hist1
plt.hist(hist_e, bins = 100, alpha = 0.5,label = "True-Regressed nu_py_2\nMean: %.2f, std: %.2f"%(hist_e.mean(), hist_e.std()))

hist1 = np.array(model({"lab_frame": x_val})[:, -1])
hist2 = np.array(y_val[:, -1])
hist_f = hist2-hist1
plt.hist(hist_f, bins = 100, alpha = 0.5,label = "True-Regressed nu_pz_2\nMean: %.2f, std: %.2f"%(hist_f.mean(), hist_f.std()))
plt.grid()
plt.legend()
plt.xlabel('Difference of Momenta component')
plt.ylabel('Occurences')
plt.xlim(-50,50)

plt.savefig('4_vect_qual_500_500_B500_R_val.png')

In [42]:
#Here verify the phi component

plt.close()
plt.close()
plt.figure()


hist1 = tf.math.atan2(model({"lab_frame": x_val})[:, i_nu1_py], model({"lab_frame": x_val})[:, i_nu1_px])
hist2 = Mom4_to_tf(nu_1.phi)[numTrain:]
hist_b = np.array(hist2-hist1)
plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_phi_1\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))
hist1 = tf.math.atan2(model({"lab_frame": x_val})[:, i_nu2_py], model({"lab_frame": x_val})[:, i_nu2_px])
hist2 = Mom4_to_tf(nu_2.phi)[numTrain:]
hist_b = np.array(hist2-hist1)
plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_phi_2\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))

plt.grid()
plt.legend()
plt.xlim(-2,2)
plt.xlabel('Difference of phi component')

plt.savefig('Phi_qual_500_500_B500_R_val.png')




In [ ]:
#Here verify the eta component

regressed_array = model({"lab_frame": x})

E_1 = norm([regressed_array[:, i_nu1_py],regressed_array[:, i_nu1_py], regressed_array[:, i_nu1_pz]])
E_2 = norm([regressed_array[:, i_nu2_py],regressed_array[:, i_nu2_py], regressed_array[:, i_nu2_pz]])


nu_1_regressed = Momentum4(E_1,regressed_array[:, i_nu1_px],regressed_array[:, i_nu1_py], regressed_array[:, i_nu1_pz])
nu_2_regressed = Momentum4(E_2,regressed_array[:, i_nu2_px],regressed_array[:, i_nu2_py], regressed_array[:, i_nu2_pz])


plt.close()
plt.close()
plt.figure()


hist1 = nu_1_regressed.p_x
hist2 = Mom4_to_tf(nu_1.p_x)#[numTrain:]
hist_b = np.array(hist2-hist1) #






plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_eta_1\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))
hist1 = nu_2_regressed.p_y
hist2 = Mom4_to_tf(nu_2.p_y)#[numTrain:]
hist_b = np.array(hist2-hist1)
plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_eta_2\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))

plt.grid()
plt.legend()
#plt.xlim(-2,2)
plt.xlabel('Difference of phi component')

plt.savefig('Phi_qual_500_500_B500_phi_all.png')


### Training 2

         
         
         

In [15]:
x = np.array([one_d(125), 
              one_d(1.776),
              #smear_px,
              #smear_py,
              #df4["metx"],
              #df4["mety"], 
              Mom4_to_tf(tau_1_vis.e),
              Mom4_to_tf(tau_1_vis.p_x),
              Mom4_to_tf(tau_1_vis.p_y),
              Mom4_to_tf(tau_1_vis.p_z),
              Mom4_to_tf(tau_2_vis.e),
              Mom4_to_tf(tau_2_vis.p_x),
              Mom4_to_tf(tau_2_vis.p_y),
              Mom4_to_tf(tau_2_vis.p_z),
              #df4["met"]
             ])
x = tf.transpose(x)

y = ref

input_1 = tf.keras.Input(shape = x.shape, name="lab_frame")

x2 = tf.keras.layers.Dense(500, activation = 'relu', name="learning")(input_1)
x3 = tf.keras.layers.Dense(300, activation = 'elu', name="learning2")(x2)
x4 = tf.keras.layers.Dense(500, activation = 'relu', name="learning3")(x3)
#x4 = tf.keras.layers.Dense(500, activation = 'relu', name="learning3")(x3)
#x5 = tf.keras.layers.Dense(500, activation = 'elu', name="learning4")(x4)
#x6 = tf.keras.layers.Dense(500, activation = 'relu', name="learning5")(x5)

output = tf.keras.layers.Dense(16, name="output")(x4)

model = tf.keras.Model(
    inputs=[input_1],
    outputs=[output],
)

#tf.keras.losses.MeanSquaredError() #common to the 4 iterations
model.compile(loss = loss_fn, optimizer = 'adam', metrics = ['mae'])#, loss_mass_Higgs, loss_mass_tau, loss_D_p])

history = model.fit(x, y, validation_split = 0.3,
    epochs=50,
    batch_size = 500)


plt.close()
plt.close()
plt.close()
plt.close()
plt.close()
plt.close()
plt.close()



#This is the 'quality check' section
fig = plt.figure('2_fig', figsize=(10,10), frameon = True)

plt.title('Quality check: 500-300-500-o,\ninputs exclude mets, output exclude met. Batch: 500')
ax = fig.add_subplot(2,1,1)

hist1 = np.array(model({"lab_frame": x})[:, -6])
hist2 = np.array(y[:, -6])
hist_a = hist2-hist1
plt.hist(hist_a, bins = 100, alpha = 0.5,label = "True-Regressed nu_px_1\nMean: %.2f, std: %.2f"%(hist_a.mean(), hist_a.std()))

hist1 = np.array(model({"lab_frame": x})[:, -5])
hist2 = np.array(y[:, -5])
hist_b = hist2-hist1
plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_py_1\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))

hist1 = np.array(model({"lab_frame": x})[:, -4])
hist2 = np.array(y[:, -4])
hist_c = hist2-hist1
plt.hist(hist_c, bins = 100, alpha = 0.5,label = "True-Regressed nu_pz_1\nMean: %.2f, std: %.2f"%(hist_c.mean(), hist_c.std()))
plt.grid()
plt.legend()
plt.xlim(-50,50)
plt.ylabel('Occurences')
         
         
ax = fig.add_subplot(2,1,2)

hist1 = np.array(model({"lab_frame": x})[:, -3])
hist2 = np.array(y[:, -3])
hist_d = hist2-hist1
plt.hist(hist_d, bins = 100, alpha = 0.5,label = "True-Regressed nu_px_2\nMean: %.2f, std: %.2f"%(hist_d.mean(), hist_d.std()))

hist1 = np.array(model({"lab_frame": x})[:, -2])
hist2 = np.array(y[:, -2])
hist_e = hist2-hist1
plt.hist(hist_e, bins = 100, alpha = 0.5,label = "True-Regressed nu_py_2\nMean: %.2f, std: %.2f"%(hist_e.mean(), hist_e.std()))

hist1 = np.array(model({"lab_frame": x})[:, -1])
hist2 = np.array(y[:, -1])
hist_f = hist2-hist1
plt.hist(hist_f, bins = 100, alpha = 0.5,label = "True-Regressed nu_pz_2\nMean: %.2f, std: %.2f"%(hist_f.mean(), hist_f.std()))
plt.grid()
plt.legend()
plt.xlabel('Difference of Momenta component')
plt.ylabel('Occurences')
plt.xlim(-50,50)

plt.savefig('Quality_500_300_500_1_500')

Epoch 1/50
1356/1356 [==============================] - 15s 11ms/step - loss: 1917.6036 - mae: 39.9942 - val_loss: 2610.1282 - val_mae: 43.4733
Epoch 2/50
1356/1356 [==============================] - 14s 10ms/step - loss: 1699.7272 - mae: 39.6700 - val_loss: 2597.8945 - val_mae: 43.7552
Epoch 3/50
1356/1356 [==============================] - 14s 10ms/step - loss: 1677.6960 - mae: 39.7335 - val_loss: 2549.5977 - val_mae: 43.7872
Epoch 4/50
1356/1356 [==============================] - 14s 11ms/step - loss: 1663.7881 - mae: 39.7429 - val_loss: 2485.2976 - val_mae: 43.4555
Epoch 5/50
1356/1356 [==============================] - 14s 10ms/step - loss: 1651.1306 - mae: 39.8377 - val_loss: 2494.3550 - val_mae: 43.6106
Epoch 6/50
1356/1356 [==============================] - 14s 10ms/step - loss: 1643.9099 - mae: 39.8467 - val_loss: 2494.1179 - val_mae: 43.6432
Epoch 7/50
1356/1356 [==============================] - 14s 10ms/step - loss: 1634.3356 - mae: 39.8797 - val_loss: 2473.9553 - val_mae: 


### Traning 3

In [44]:
# Here copying the result from the KIT paper

smear_px, smear_py = one_d(0), one_d(0)   #the smearing of the detector, we don't know yet what it is

ref = [#smear_px,py                      #0
       one_d(1.776),                      #1
       df4["metx"],                   #2
       df4["mety"],                   #3
       Mom4_to_tf(tau_1_vis.e),       #4
       Mom4_to_tf(tau_1_vis.p_x),     #5
       Mom4_to_tf(tau_1_vis.p_y),     #6
       Mom4_to_tf(tau_1_vis.p_z),     #7
       Mom4_to_tf(tau_2_vis.e),       #8
       Mom4_to_tf(tau_2_vis.p_x),     #9 
       Mom4_to_tf(tau_2_vis.p_y),     #10
       Mom4_to_tf(tau_2_vis.p_z),     #11
       one_d(125),                    #12
       #Mom4_to_tf(nu_1.e),            #13       corresponding to          #0
       Mom4_to_tf(nu_1.p_x),          #14                                 #1
       Mom4_to_tf(nu_1.p_y),          #15                                 #2
       Mom4_to_tf(nu_1.p_z),          #16                                 #3
       #Mom4_to_tf(nu_2.e),            #17                                 #4
       Mom4_to_tf(nu_2.p_x),          #18                                 #5
       Mom4_to_tf(nu_2.p_y),          #19                                 #6
       Mom4_to_tf(nu_2.p_z),          #20                                 #7
]


#i_smear_px = 0
i_tau_mass = 0
i_smeared_met_px = 1
i_smeared_met_py = 2
i_tau1_e = 3
i_tau1_px = 4
i_tau1_py = 5
i_tau1_pz = 6
i_tau2_e = 7
i_tau2_px = 8
i_tau2_py = 9
i_tau2_pz = 10
i_gen_mass = 11
#i_nu1_e = 12
i_nu1_px = 12
i_nu1_py = 13
i_nu1_pz = 14
#i_nu2_e = 16
i_nu2_px = 15
i_nu2_py = 16
i_nu2_pz = 17


m_tau_squared = tf.transpose(one_d(1.776)**2)

ref = tf.transpose(ref)

def loss_D_p (y_true, y_pred):
    #calculting the difference between the the components, need to add the smearing of detector eventually
    target_components = [i_nu1_px, i_nu1_py, i_nu1_pz, i_nu2_px, i_nu2_py, i_nu2_pz]
    target_components_diff_list = []
    for i in target_components: target_components_diff_list.append((y_true[:,i]-y_pred[:,i])**2)
    dxyz = 0
    for d in target_components_diff_list: dxyz+=d
    return dxyz

def energy_nu (y_true, y_pred, number):
    if number == 1:
        return tf.sqrt(y_pred[:, i_nu1_px]**2 + y_pred[:, i_nu1_py]**2 + y_pred[:, i_nu1_pz]**2)
    if number == 2:
        return tf.sqrt(y_pred[:, i_nu2_px]**2 + y_pred[:, i_nu2_py]**2 + y_pred[:, i_nu2_pz]**2)

def loss_mass_tau(y_true, y_pred):
    #now we try only to use the y_pred for neutrino info, this is I guess their way of 
    #only training for neutrino info whilst keeping nice structure
    #we are always assuming m=0
    # note, we are taking y_tau as exact, we could choose not to...
    E1 = y_true[:, i_tau1_e] + energy_nu(y_true, y_pred, 1) 
    P1_squared = (y_true[:, i_tau1_px] + y_pred[:, i_nu1_px])**2 + (y_true[:, i_tau1_py] + y_pred[:, i_nu1_py])**2 + (y_true[:, i_tau1_pz] + y_pred[:, i_nu1_pz])**2
    R1 = (E1**2 - P1_squared - y_true[:, i_tau_mass]**2)/(y_true[:, i_gen_mass])**2
    
    E2 = y_true[:, i_tau2_e] + energy_nu(y_true, y_pred, 2) 
    P2_squared = (y_true[:, i_tau2_px] + y_pred[:, i_nu2_px])**2 + (y_true[:, i_tau2_py] + y_pred[:, i_nu2_py])**2 + (y_true[:, i_tau2_pz] + y_pred[:, i_nu2_pz])**2
    R2 = (E2**2 - P2_squared - y_true[:, i_tau_mass]**2)/(y_true[:, i_gen_mass])**2
    return tf.math.abs(R1) + tf.abs(R2)


def loss_mass_Higgs(y_true, y_pred):
    EH = y_true[:, i_tau1_e] + energy_nu(y_true, y_pred, 1) + y_true[:, i_tau2_e] + energy_nu(y_true, y_pred, 2)
    px_H = y_true[:, i_tau1_px] + y_true[:, i_tau2_px] + y_pred[:, i_nu1_px] + y_pred[:, i_nu2_px]
    py_H = y_true[:, i_tau1_py] + y_true[:, i_tau2_py] + y_pred[:, i_nu1_py] + y_pred[:, i_nu2_py]
    pz_H = y_true[:, i_tau1_pz] + y_true[:, i_tau2_pz] + y_pred[:, i_nu1_pz] + y_pred[:, i_nu2_pz]
    
    return tf.abs((EH**2-px_H**2-py_H**2-pz_H**2 - y_true[:, i_gen_mass]**2)/(y_true[:,i_gen_mass])**2)
    

print(loss_mass_tau(ref,ref))
print(loss_mass_Higgs(ref,ref))


def loss_fn(y_true, y_pred):

    dxyz = loss_D_p(y_true, y_pred)
    #dmet = loss_dmet(y_true, y_pred)
    #dPTtaus = loss_dPTtaus(y_true, y_pred)
    dtau = loss_mass_tau(y_true, y_pred)
    dHiggs = loss_mass_Higgs(y_true, y_pred)
    #dM = loss_dM_had(y_true, y_pred)

    return dxyz + dtau + dHiggs


x = np.array([one_d(125), 
              one_d(1.776),
              #smear_px,
              #smear_py,
              df4["metx"],
              df4["mety"], 
              Mom4_to_tf(tau_1_vis.e),
              Mom4_to_tf(tau_1_vis.p_x),
              Mom4_to_tf(tau_1_vis.p_y),
              Mom4_to_tf(tau_1_vis.p_z),
              Mom4_to_tf(tau_2_vis.e),
              Mom4_to_tf(tau_2_vis.p_x),
              Mom4_to_tf(tau_2_vis.p_y),
              Mom4_to_tf(tau_2_vis.p_z),
              df4["met"],

             ])
x = tf.transpose(x)

y = ref


input_1 = tf.keras.Input(shape = x.shape, name="lab_frame")

x2 = tf.keras.layers.Dense(500, activation = 'relu', name="learning")(input_1)
x3 = tf.keras.layers.Dense(500, activation = 'elu', name="learning3")(x2)
#x4 = tf.keras.layers.Dense(100, activation = 'elu', name="learning3")(x3)
#x4 = tf.keras.layers.Dense(500, activation = 'relu', name="learning3")(x3)
#x5 = tf.keras.layers.Dense(500, activation = 'elu', name="learning4")(x4)
#x6 = tf.keras.layers.Dense(500, activation = 'relu', name="learning5")(x5)

output = tf.keras.layers.Dense(18, name="output")(x3)

model = tf.keras.Model(
    inputs=[input_1],
    outputs=[output],
)

model.summary

#tf.keras.losses.MeanSquaredError() #common to the 4 iterations
model.compile(loss = loss_fn, optimizer = 'adam', metrics = ['mae'])#, loss_mass_Higgs, loss_mass_tau, loss_D_p])

history = model.fit(x, y, validation_split = 0.3,
    epochs=50,
    batch_size = 500)



#This is the 'quality check' section
fig = plt.figure('2_fig', figsize=(10,10), frameon = True)

plt.title('Quality check: 300-300-o,\ninputs include mets, output include met. Batch: 500')
ax = fig.add_subplot(2,1,1)

hist1 = np.array(model({"lab_frame": x})[:, -6])
hist2 = np.array(y[:, -6])
hist_a = hist2-hist1
plt.hist(hist_a, bins = 100, alpha = 0.5,label = "True-Regressed nu_px_1\nMean: %.2f, std: %.2f"%(hist_a.mean(), hist_a.std()))

hist1 = np.array(model({"lab_frame": x})[:, -5])
hist2 = np.array(y[:, -5])
hist_b = hist2-hist1
plt.hist(hist_b, bins = 100, alpha = 0.5,label = "True-Regressed nu_py_1\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))

hist1 = np.array(model({"lab_frame": x})[:, -4])
hist2 = np.array(y[:, -4])
hist_c = hist2-hist1
plt.hist(hist_c, bins = 100, alpha = 0.5,label = "True-Regressed nu_pz_1\nMean: %.2f, std: %.2f"%(hist_c.mean(), hist_c.std()))
plt.grid()
plt.legend()
plt.xlim(-50,50)
plt.ylabel('Occurences')
         
         
ax = fig.add_subplot(2,1,2)

hist1 = np.array(model({"lab_frame": x})[:, -3])
hist2 = np.array(y[:, -3])
hist_d = hist2-hist1
plt.hist(hist_d, bins = 100, alpha = 0.5,label = "True-Regressed nu_px_2\nMean: %.2f, std: %.2f"%(hist_d.mean(), hist_d.std()))

hist1 = np.array(model({"lab_frame": x})[:, -2])
hist2 = np.array(y[:, -2])
hist_e = hist2-hist1
plt.hist(hist_e, bins = 100, alpha = 0.5,label = "True-Regressed nu_py_2\nMean: %.2f, std: %.2f"%(hist_e.mean(), hist_e.std()))

hist1 = np.array(model({"lab_frame": x})[:, -1])
hist2 = np.array(y[:, -1])
hist_f = hist2-hist1
plt.hist(hist_f, bins = 100, alpha = 0.5,label = "True-Regressed nu_pz_2\nMean: %.2f, std: %.2f"%(hist_f.mean(), hist_f.std()))
plt.grid()
plt.legend()
plt.xlabel('Difference of Momenta component')
plt.ylabel('Occurences')
plt.xlim(-50,50)

plt.savefig('Quality_300_300_include_met_500')

tf.Tensor(
[1.22657775e-05 8.39220302e-05 7.18750016e-06 ... 6.93282782e-05
 1.77967231e-05 9.17172201e-06], shape=(571700,), dtype=float32)
tf.Tensor([0.02934769 0.20486112 0.01050362 ... 0.12745337 0.02100937 0.00848406], shape=(571700,), dtype=float32)
Epoch 1/50
801/801 [==============================] - 10s 12ms/step - loss: 1329.0811 - mae: 38.1916 - val_loss: nan - val_mae: nan
Epoch 2/50
582/801 [====================>.........] - ETA: 2s - loss: 1029.9915 - mae: 37.2705

KeyboardInterrupt: 

In [59]:
hist1 = np.array(model({"lab_frame": x})[:, -2])

print(len(model({"lab_frame": x})[1]))
hist2 = np.array(y[:, -2])
need = "nu_1_E"
dd = 0

def frac(d = -2):
    difference = y[:, 0]-model({"lab_frame": x})[:, 0]
    difference = np.reshape(difference, [-1])
    l = np.where(abs(difference)<=10**(d),1,0)
    print(float(float(np.sum(l))/len(l)), d)
    return float(float(np.sum(l))/len(l))

plt.figure()
plt.hist(hist1, bins = 100, alpha = 0.5,label = "NN %s component : fraction($\Delta$<$10^{%.1f}$)=%.3f "%(need, dd, frac(dd)))
plt.hist(hist2, bins = 100, alpha = 0.5,label = 'True %s - Features: phi_CP_1 (fixed)'%need)      
plt.ylabel("Frequency", fontsize = 'x-large')
plt.xlabel("%s"%(need), fontsize = 'x-large')
plt.grid()
#plt.xlim(0,400)
plt.legend()#prop = {'size', 10})
plt.savefig('neutrino_next_2_300_2')

16
0.1561270904944474 0


In [ ]:
class Calculation:
    """
    Class for calculating the aco_angle variables
    """
    def __init__(self, df):
        #this is the function doing all the calculations manually just takes as input the dataframe
        #The different *initial* 4 vectors, (E,px,py,pz)
        self.pi_1 = np.array([df["pi_E_1"],df["pi_px_1"],df["pi_py_1"],df["pi_pz_1"]])
        self.pi_2 = np.array([df["pi_E_2"],df["pi_px_2"],df["pi_py_2"],df["pi_pz_2"]])

        self.pi0_1 = np.array([df["pi0_E_1"],df["pi0_px_1"],df["pi0_py_1"],df["pi0_pz_1"]])
        self.pi0_2 = np.array([df["pi0_E_2"],df["pi0_px_2"],df["pi0_py_2"],df["pi0_pz_2"]])

        #Charged and neutral pion momenta
        self.pi_1_4Mom = Momentum4(df["pi_E_1"],df["pi_px_1"],df["pi_py_1"],df["pi_pz_1"])
        self.pi_2_4Mom = Momentum4(df["pi_E_2"],df["pi_px_2"],df["pi_py_2"],df["pi_pz_2"])

        #Same for the pi0
        self.pi0_1_4Mom = Momentum4(df["pi0_E_1"],df["pi0_px_1"],df["pi0_py_1"],df["pi0_pz_1"])
        self.pi0_2_4Mom = Momentum4(df["pi0_E_2"],df["pi0_px_2"],df["pi0_py_2"],df["pi0_pz_2"])

        self.impact_param_1 = Momentum4(np.zeros(len(df["ip_x_1"])),df["ip_x_1"],df["ip_y_1"],df["ip_z_1"])
        self.impact_param_2 = Momentum4(np.zeros(len(df["ip_x_2"])),df["ip_x_2"],df["ip_y_2"],df["ip_z_2"])

        #comment or uncomment depending on which aco_angle you want 
        #self.pi0_1_4Mom = self.impact_param_1
        #self.pi0_2_4Mom = self.impact_param_2

        #This is the COM frame of the two charged pions w.r.t. which we'll boost
        self.ref_COM_4Mom = Momentum4(self.pi_1_4Mom+self.pi_2_4Mom)
        boost = Momentum4(self.ref_COM_4Mom[0], -self.ref_COM_4Mom[1], -self.ref_COM_4Mom[2], -self.ref_COM_4Mom[3])
        
        
        boost = -self.ref_COM_4Mom
        #energies=[df4["pi_E_1"],df4["pi_E_2"],df4["pi0_E_1"],df4["pi0_E_2"]]

        #Lorentz boost everything in the ZMF of the two charged pions
        self.pi0_1_4Mom_star = self.pi0_1_4Mom.boost_particle(boost)
        self.pi0_2_4Mom_star = self.pi0_2_4Mom.boost_particle(boost)

        #Lorentz boost everything in the ZMF of the two neutral pions
        self.pi_1_4Mom_star = self.pi_1_4Mom.boost_particle(boost)
        self.pi_2_4Mom_star = self.pi_2_4Mom.boost_particle(boost)


        #calculating the perpependicular component
        pi0_1_3Mom_star_perp=cross_product(self.pi0_1_4Mom_star[1:], self.pi_1_4Mom_star[1:])
        pi0_2_3Mom_star_perp=cross_product(self.pi0_2_4Mom_star[1:], self.pi_2_4Mom_star[1:])

        #Now normalise:
        pi0_1_3Mom_star_perp=pi0_1_3Mom_star_perp/norm(pi0_1_3Mom_star_perp)
        pi0_2_3Mom_star_perp=pi0_2_3Mom_star_perp/norm(pi0_2_3Mom_star_perp)

        self.pi0_1_4Mom_star_perp = [self.pi0_1_4Mom_star[0], pi0_1_3Mom_star_perp[0], 
                                     pi0_1_3Mom_star_perp[1], pi0_1_3Mom_star_perp[2]]

        self.pi0_2_4Mom_star_perp = [self.pi0_1_4Mom_star[0], pi0_2_3Mom_star_perp[0], 
                                     pi0_2_3Mom_star_perp[1], pi0_2_3Mom_star_perp[2]]

        #Calculating phi_star
        self.phi_CP_unshifted = np.arccos(dot_product(pi0_1_3Mom_star_perp,pi0_2_3Mom_star_perp))
        
        print(self.phi_CP_unshifted[:10],'This is phi_CP')

        self.phi_CP = self.phi_CP_unshifted
        
        print(pi0_1_3Mom_star_perp[:,23], 'this is pi0_1_3mom')

        #The energy ratios
        self.y_T = np.array(df['y_1_1']*df['y_1_2'])

        #The O variable
        cross = np.array(np.cross(pi0_1_3Mom_star_perp.transpose(),pi0_2_3Mom_star_perp.transpose()).transpose())
        self.bigO = dot_product(self.pi_2_4Mom_star[1:],cross)
        
        print(self.bigO[:10], '\n this is big0')

        #perform the shift w.r.t. O* sign
        
        
        #phi_CP=np.where(self.bigO>=0, 2*np.pi-self.phi_CP_unshifted, self.phi_CP_unshifted)
        self.phi_CP_1 = np.where(self.bigO>=0, 2*np.pi-self.phi_CP_unshifted, self.phi_CP_unshifted)
        
        print(self.phi_CP_1[:10], '\n this is after first shft')

       # self.phi_CP_2 = np.where(self.y_T<=0, self.phi_CP+np.pi, self.phi_CP-np.pi)

        #additionnal shift that needs to be done do see differences between odd and even scenarios, with y=Energy ratios
        self.phi_CP = np.where(self.y_T>=0, np.where(self.phi_CP_1<np.pi, self.phi_CP_1+np.pi, self.phi_CP_1-np.pi), self.phi_CP_1)
        
        self.df = df
        
        print(self.phi_CP[:10], 'this is full')
        
        self.y = df["aco_angle_1"]
    
    def checks(self):

        target = [self.df["aco_angle_1"]]#self.df["aco_angle_7"]]
        y = tf.transpose(tf.convert_to_tensor(target, dtype=np.float32))

        inputs = [self.pi0_1_4Mom, self.pi_1_4Mom, self.pi0_2_4Mom, self.pi_2_4Mom]
        x = tf.convert_to_tensor(inputs, dtype=np.float32)
        x = tf.transpose(x, [2, 0, 1])
        
        k = tf.convert_to_tensor([
                          self.impact_param_1[0], self.impact_param_1[1], self.impact_param_1[2], self.impact_param_1[3],
                          #self.pi0_1_4Mom[0], self.pi0_1_4Mom[1], self.pi0_1_4Mom[2], self.pi0_1_4Mom[3],
                          self.pi_1_4Mom[0], self.pi_1_4Mom[1], self.pi_1_4Mom[2], self.pi_1_4Mom[3],
                          #self.pi0_2_4Mom[0], self.pi0_2_4Mom[1], self.pi0_2_4Mom[2], self.pi0_2_4Mom[3],
                          self.impact_param_2[0], self.impact_param_2[1], self.impact_param_2[2], self.impact_param_2[3],
                          self.pi_2_4Mom[0], self.pi_2_4Mom[1], self.pi_2_4Mom[2], self.pi_2_4Mom[3]],
                         dtype=np.float32)

# the extra info we are giving
        l = tf.convert_to_tensor([self.y_T], dtype=np.float32)

        return x,y,k,l

In [57]:
print (len(x[1]))

10


In [57]:
tau_1 = nu_1 + tau_1_vis
tau_2 = nu_2 + tau_2_vis

plt.figure()
plt.hist(tau_1.m,  bins = 1000, alpha = 0.5, label = 'dot product |tau_1_vis| and |nu_1|\nmean=%.2f, std=%.2f'% (np.array(tau_1.m).mean(), np.array(tau_1.m).std()))
plt.hist(np.array(tau_2.m),  bins = 1000, alpha = 0.5, label = 'dot product |tau_2_vis| and |nu_2|\nmean=%.2f, std=%.2f'% (np.array(tau_2.m).mean(), np.array(tau_2.m).std()))
plt.grid()
plt.legend()
plt.xlim(0,6)
plt.ylabel('Occurencies', fontsize = 'x-large')
plt.xlabel('Mass of sum of rho decay products and tau neutrino', fontsize = 'large')
plt.title('Checking that tau neutrino and visible decay products\nsum up to tau (RM9999)', fontsize = 'xx-large', weight = 'bold')
#plt.show()
plt.savefig('sum_to_tau')

In [60]:
Higgs = tau_1 + tau_2 
plt.figure()
plt.hist(Higgs.m,  bins = 1000, alpha = 0.5, label = 'dot product |tau_1_vis| and |nu_1|\nmean=%.2f, std=%.2f'% (np.array(Higgs.m).mean(), np.array(Higgs.m).std()))
plt.grid()
plt.legend()
#plt.xlim(0,6)
plt.ylabel('Occurencies', fontsize = 'x-large')
plt.xlabel('Mass of the sum of the two taus', fontsize = 'large')
plt.title('Checking that the two taus\nsum up to Higgs (RM9999)', fontsize = 'xx-large', weight = 'bold')
#plt.show()
plt.savefig('sum_to_Higgs')

<ipython-input-60-71f6fbb4e571>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [5]:
#check of the colinearity approximation

#Next up: colinarity, tau mass (Kingsley checked) and then Higgs. 

dot_prod_1 = np.einsum('ia,ia->a',tau_1_vis[1:, ...], nu_1[1:, ...])
dot_prod_1 = dot_prod_1/(norm(tau_1_vis[1:, ...])* norm(nu_1[1:, ...]))

dot_prod_2 = np.einsum('ia,ia->a',tau_2_vis[1:, ...], nu_2[1:, ...])
dot_prod_2 = dot_prod_2/(norm(tau_2_vis[1:, ...])* norm(nu_2[1:, ...]))

print(dot_prod_1)
print(dot_prod_2[:10])

[0.99919824 0.99922621 0.99976824 ... 0.99905886 0.99952941 0.99982143]
[0.99770581 0.99983855 0.99924261 0.98171425 0.9989211  0.99978589
 0.99934793 0.99860659 0.99942672 0.99745414]


In [7]:
plt.figure()
plt.hist(dot_prod_1,  bins = 1000, alpha = 0.5, label = 'dot product |tau_1_vis| and |nu_1|\nmean=%.2f, std=%.2f'% (dot_prod_1.mean(), dot_prod_1.std()))
plt.hist(dot_prod_2,  bins = 1000, alpha = 0.5, label = 'dot product |tau_2_vis| and |nu_2|\nmean=%.2f, std=%.2f'% (dot_prod_2.mean(), dot_prod_2.std()))
plt.grid()
plt.legend()
plt.xlim(0.95,1.05)
plt.ylabel('Occurencies', fontsize = 'x-large')
plt.xlabel('Dot product between rho decay products and tau neutrino', fontsize = 'large')
plt.title('Checking the colinarity approximation between\ntau neutrino and visible decay products', fontsize = 'xx-large', weight = 'bold')

plt.savefig('Colinearity')

In [36]:
#Check the colinear approximation ? can calculate the dot product between the visible decay products and the nus
check_x_1 = np.array(nu_1.p_x-tau_1_vis.p_x)#dot_product(tau_1_vis[1:]/norm(tau_1_vis[1:]), nu_1[1:]/norm(nu_1[1:])))
check_x_2 = np.array(nu_2.p_x-tau_2_vis.p_x)#dot_product(tau_2_vis[1:]/norm(tau_2_vis[1:]), nu_2[1:]/norm(nu_2[1:])))
check_y_1 = np.array(nu_1.p_y-tau_1_vis.p_y)
check_y_2 = np.array(nu_2.p_y-tau_2_vis.p_y)


check_z_1 = []
check_z_2 = []

for i in range (len(nu_1.p_z)):
    if nu_1.p_z[i] != 9999:
        check_z_1.append(nu_1.p_z[i]-tau_1_vis.p_z[i])
    if nu_2.p_z[i] != 9999:
        check_z_2.append(nu_2.p_z[i]-tau_2_vis.p_z[i])

check_z_1 = np.array(check_z_1)
check_z_2 = np.array(check_z_2)

plt.figure()
plt.hist(tau_1_vis.p_x - nu_1.p_x, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_px and nu_1_px\nmean=%.2f, std=%.2f'% (check_x_1.mean(),check_x_1.std()))
plt.hist(tau_2_vis.p_x - nu_2.p_x, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_px and nu_2_px\nmean=%.2f, std=%.2f'% (check_x_2.mean(),check_x_2.std()))

plt.title('Sanity check - difference between \nsum(pions) and neutrino momenta components', fontsize = 'xx-large', weight = 'bold')
plt.xlabel('Difference between momenta components', fontsize = 'x-large')
plt.ylabel('Frequency', fontsize = 'x-large')
plt.grid()
plt.xlim(-250,250)
plt.legend()
plt.savefig('Check_angles_remove_x')

plt.figure()

plt.hist(tau_1_vis.p_y - nu_1.p_y, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_py and nu_1_py\nmean=%.2f, std=%.2f'% (check_y_1.mean(),check_y_1.std()))
plt.hist(tau_2_vis.p_y - nu_2.p_y, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_py and nu_2_py\nmean=%.2f, std=%.2f'% (check_y_2.mean(),check_y_2.std()))
plt.title('Sanity check - difference between \nsum(pions) and neutrino momenta components', fontsize = 'xx-large', weight = 'bold')
plt.xlabel('Difference between momenta components', fontsize = 'x-large')
plt.ylabel('Frequency', fontsize = 'x-large')
plt.grid()
plt.xlim(-250,250)
plt.legend()
plt.savefig('Check_angles_remove_y')


plt.figure()

plt.hist(check_z_1, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_pz and nu_1_pz\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_z_1.mean(),check_z_1.std()))
plt.hist(check_z_2, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_pz and nu_2_pz\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_z_2.mean(),check_z_2.std()))


plt.title('Sanity check - difference between \nsum(pions) and neutrino momenta components', fontsize = 'xx-large', weight = 'bold')
plt.xlabel('Difference between momenta components', fontsize = 'x-large')
plt.ylabel('Frequency', fontsize = 'x-large')
plt.grid()
plt.xlim(-250,250)
plt.legend()
plt.savefig('Check_angles_remove_z')


#plt.show()

In [35]:
check_E_1 = []#dot_product(tau_1_vis[1:]/norm(tau_1_vis[1:]), nu_1[1:]/norm(nu_1[1:])))
check_E_2 = []#dot_product(tau_2_vis[1:]/norm(tau_2_vis[1:]), nu_2[1:]/norm(nu_2[1:])))

for i in range (len(nu_1.e)):
    if nu_1.e[i]!=9999:
        check_E_1.append(nu_1.e[i]-tau_1_vis.e[i])
    if nu_2.e[i]!=9999:
        check_E_2.append(nu_2.e[i]-tau_2_vis.e[i])

check_E_1 = np.array(check_E_1)
check_E_2 = np.array(check_E_2)
plt.figure()
plt.hist(check_E_1, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_E and nu_1_E\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_E_1.mean(),check_E_1.std()))
plt.hist(check_E_2, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_E and nu_2_E\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_E_2.mean(),check_E_2.std()))


plt.title('Sanity check - difference between \nsum(pions) and neutrino energy components', fontsize = 'xx-large', weight = 'bold')
plt.xlabel('Difference between energies', fontsize = 'x-large')
plt.ylabel('Frequency', fontsize = 'x-large')
plt.grid()
plt.xlim(-500,500)
plt.legend()
plt.savefig('Check_energies')

In [37]:
sum_nu = nu_1 + nu_2

met = np.array(df4["met"])
sum_energies=[]
met_ref = []
x = [0,500]

for i in range (len(nu_1.e)):
    if nu_1.e[i]!=9999 and nu_2.e[i]!=9999:
        sum_energies.append(sum_nu.e[i])
        met_ref.append(met[i])

plt.figure()

plt.plot(met_ref, sum_energies, 'gx')
plt.plot(x,x, 'k--', label = 'y=x')
plt.xlabel("Missing transverse energy", fontsize = 'x-large')
plt.ylabel("Energy of the sum of the neutrinos", fontsize = 'x-large')
plt.title("Sanity check, removing 9999 \n gen level nu energies and met", fontsize = 'xx-large', weight = 'bold')
plt.grid()
plt.legend()
plt.xlim(0,500)
plt.savefig('Compare_nu_E_to_met')

<ipython-input-37-75e86c0dc241>:29: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('Compare_nu_E_to_met')


In [120]:
x_ref = np.array([0, 6])
x_ref2 = [6,0]

gen_phitt = np.array(df4["gen_phitt"][:2000])*2*np.pi/180+np.pi

plt.figure()
plt.plot(x_ref, x_ref, '--', label = 'x=y')
plt.plot(x_ref, x_ref2, '--', label = 'y=-x')

plt.hist2d(df4["aco_angle_1"], df4["gen_phitt"], bins=50)
#plt.plot(df4["aco_angle_1"][:2000], gen_phitt, 'bx', label='aco_angle_1')
#plt.plot(df4["aco_angle_5"][:100], gen_phitt, 'gx', label='aco_angle_5')
#plt.plot(df4["aco_angle_6"][:100], gen_phitt, 'rx', label='aco_angle_6')
#plt.plot(df4["aco_angle_7"][:100], gen_phitt, 'kx', label='aco_angle_7')
#plt.xlabel("aco_angles", fontsize = 'x-large')
#plt.ylabel("gen_phitt, (rad & shifted)", fontsize = 'x-large')
plt.title("Sanity check,\n gen_phitt against aco angles", fontsize = 'xx-large', weight = 'bold')
#plt.grid()
#plt.legend()
plt.savefig('Gen_phitt-aco_1')

<ipython-input-120-d78cf37700c1>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [61]:
plt.close()
plt.close()
plt.close()
plt.close()

## Simple NN, no custom loss fn

In [33]:
plt.close()
plt.close()
plt.close()
fig = plt.figure(figsize=(10,10), frameon = True)

plt.title('Quality check: 300-300-300-o,\n Only regress phi Batch: 500')
#ax = fig.add_subplot(2,1,1)

hist1 = np.array(model({"lab_frame": x})[:, 0])
hist2 = np.array(y[:, 0])
hist_a = hist2-hist1
plt.hist(hist_a, bins = 1000, alpha = 0.5,label = "True-Regressed nu_phi_1\nMean: %.2f, std: %.2f"%(hist_a.mean(), hist_a.std()))

# hist1 = np.array(model({"lab_frame": x})[:, 1])
# hist2 = np.array(y[:, 1])
# hist_b = hist2-hist1
# plt.hist(hist_b, bins = 1000, alpha = 0.5,label = "True-Regressed nu_phi_2\nMean: %.2f, std: %.2f"%(hist_b.mean(), hist_b.std()))

plt.grid()
plt.legend()
plt.xlim(-2,2)
plt.ylabel('Occurences')
plt.xlabel('Difference in phi')

plt.savefig('Regress_only_phi_C4.png')
         

Below you can find all the tranings with the custom loss function and the full 4 vector of the neutrinos, here for book keeping.

In [48]:
Mom4_to_tf(tau_1_vis.e) - 4

<tf.Tensor: shape=(968192,), dtype=float32, numpy=
array([51.463295, 61.751213, 80.776825, ..., 44.725338, 57.42529 ,
       80.36569 ], dtype=float32)>

In [16]:
# Here start checking the colinearity method

#first verify the spread of phi_nu compared to phi_vis
hist = np.array(nu_1.phi - tau_1_vis.phi)
hist1 = np.array(nu_2.phi - tau_2_vis.phi)

plt.close()
plt.close()
plt.figure()
plt.hist(hist, bins = 1000, alpha=0.5, label = 'Difefrence between nu_phi_1 and vis_phi_1\nMean:%.2f, Std:%.2f'%(hist.mean(), hist.std()))
plt.hist(hist1, bins = 1000, alpha=0.5, label = 'Difefrence between nu_phi_2 and vis_phi_2\nMean:%.2f, Std:%.2f'%(hist1.mean(), hist1.std()))
plt.legend()
plt.xlim(-1,1)
plt.grid()
plt.ylabel('Occurencies')
plt.title('Check for phi')
plt.xlabel('nu_phi - vis_phi')
plt.savefig('Diff_nu_vis_phi.png')

In [7]:
#quadratic distance to target - regress both at the same time, will be required for making sense out of the met

#try to first work with aplha, only 1 variable - easier
# y_1 is the first regressed value, I want them as [[],[]]
#D_target = (apha_1 - y_1)**2 + (apha_2 - y_2)**2 

def tau_mass_dist_1(y, y_pred):
    global tau_1_vis  
    
    y = tf.transpose(y)
    y_pred = tf.transpose(y_pred)
    
    
    sum_energy = tf.transpose(tf.convert_to_tensor(tau_1_vis.e, dtype = 'float32'))+ tf.transpose(y_pred)[0] * tf.transpose(tf.convert_to_tensor(tau_1_vis.p, dtype = 'float32'))
    
    sum_p = (1+ tf.transpose(y_pred)[0]) * tf.transpose(tf.convert_to_tensor(tau_1_vis.p, dtype = 'float32'))
    
    
    sum_energy_1 = tf.transpose(tf.convert_to_tensor(tau_1_vis.e, dtype = 'float32'))+ y[0] * tf.transpose(tf.convert_to_tensor(tau_1_vis.p, dtype = 'float32'))
    
    sum_p_1 = (1+ tf.transpose(y[0])) * tf.transpose(tf.convert_to_tensor(tau_1_vis.p, dtype = 'float32'))
    
    
    
     
    m_tau =  tf.transpose(tf.convert_to_tensor(np.ones(sum_p.shape) * 1.77, dtype = 'float32')) 
    
    return tf.math.abs(sum_energy_1 + sum_energy)#tf.math.abs(sum_energy**2 - sum_p**2 - m_tau**2)  #tf.math.mod(sum_energy**2 - sum_p**2 - m_tau**2)


def loss_fn(y, y_pred):
#     y = tf.transpose(y)
#     y_pred = tf.transpose(y_pred)
    return tf.convert_to_tensor((y[0] - y_pred[0])**2 + (y[1] - y_pred[1])**2 + tau_mass_dist_1(y, y_pred))#, dtype = np.float32)


#sum_energy**2 - sum_p**2 -

# + y[1] * tf.convert_to_tensor(tau_2_vis.p)
    
    

print (tau_mass_dist_1([alpha_1, alpha_2],[alpha_1, alpha_2]))


InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Mul]